# HousingPrices
Implement from scratch the ridge regression algorithm for regression with square loss.

Apply the algorithm to the prediction of the label medianHouseValue in this dataset.

Study the dependence of the cross-validated risk estimate on the parameter alpha of ridge regression.

Try using PCA to improve the risk estimate.

Optionally, use nested cross-validated risk estimates to remove the need of choosing the parameter.

Here is a description of the attributes in the dataset:
1. longitude: A measure of how far west a house is; a higher value is farther west
2. latitude: A measure of how far north a house is; a higher value is farther north
3. housingMedianAge: Median age of a house within a block; a lower number is a newer building
4. totalRooms: Total number of rooms within a block
5. totalBedrooms: Total number of bedrooms within a block
6. population: Total number of people residing within a block
7. households: Total number of households, a group of people residing within a home unit, for a block
8. medianIncome: Median income for households within a block of houses (measured in tens of thousands of US Dollars)
9. medianHouseValue: Median house value for households within a block (measured in US Dollars)
10. oceanProximity: Location of the house w.r.t ocean/sea

Note: The dataset has an attribute with missing values and an attribute with categorical values. Find a way of handling these anomalies and justify your choice.


## License
Copyright 2020 (c) Anna Olena Zhab'yak, Michele Maione. All rights reserved.

Licensed under the [MIT](LICENSE) License.

----------------
# Ridge regression
This is a regression technique wich allows to deal with the multicollinearity of the features and try to reduce the overfitting of the data. 
minimize two elements, the RSS and the objective function 

In ridge regression two elements are optimized , the RSS and the objective function

$ \lVert y - Xw \rVert ^2_2 + \alpha \lVert w \rVert^2_2 $

It is a variation of the Linear regression with a tunning parameter alpha to control model complexity.
need to do a trade off between variance and bias
1. the inhability for the model to capture the true relationship is called bias. How well the model fits our data 
2. how well model does on a completely new dataset. 

use the hyper parameter called alpha , to add a bias , prevent the model from real, it will have better accurancy on the testing set, it means it will not overfit in the training set.  

## Libraries 

For this analysis, we will use a few libraries for managing and transform the database as well as to implement the ridge regression. First, let's download the libraries 

In [1]:
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import learning_curve, validation_curve, cross_val_score, train_test_split, KFold, GridSearchCV

from LinearRegression.RidgeRegression.cholesky import Cholesky
from LinearRegression.RidgeRegression.svd import SVD

ImportError: No module named numpy

----------------
## Dataset analysis
Open the database, check the structure, show the head, take a look of its structure, the descriptive statistics and manage the columns.

Create the constants

The target value is the median_house_value which is predicted based on the features of different houses.

### Mean squared error
The mean squared error function computes mean square error, a risk metric corresponding to the expected value of the squared (quadratic) error or loss.

### R²
The coefficient R² is defined as $ 1 - {u \over v} $, where u is the residual sum of squares $ \sum (y - y')^2 $ and v is the total sum of squares $ \sum (y - \bar{y})^2 $.

The best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse).

A constant model that always predicts the expected value of y, disregarding the input features, would get a R² score of 0.0.

In [ ]:
RMS = 'neg_mean_squared_error'
R2 = 'r2'

#scoring = R2
#scoring_neg = 1
#scoring_label = 'R²'

scoring = RMS
scoring_neg = -1
scoring_label = 'MSE'

In [ ]:
nested_cross_validation_trials = 10

fit_intercept = True
normalize = True

shuffleDataSet = False

cpu = -1

Valori di ɑ usati

In [ ]:
alphas = numpy.linspace(1e-5, 1, 20)
alphas

Create the constants

In [ ]:
column_to_predict = 'median_house_value'
categories_columns = ['ocean_proximity']
numerics_columns = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income"]

In [ ]:
data_frame = pandas.read_csv(filepath_or_buffer='cal-housing.csv')

# shuffle all DB
data_frame = data_frame.sample(frac=1).reset_index(drop=True)

In [ ]:
data_frame.info()

In [ ]:
data_frame.describe()

In [ ]:
data_frame.head()

Here is the distribution of the values of the target variable "median_house_value".
It looks like a normal distribution with a group of outliers on the highest value of the house.

In [ ]:
plt.figure(figsize=(15, 7))
plt.title('Distribuzione dei valori di "median_house_value"')

seaborn.distplot(data_frame[column_to_predict])

plt.grid()
plt.show()

----------------
## Data pre-processing

### Outliers

In [ ]:
outliers = data_frame[data_frame[column_to_predict] == 500001].index

print(f'There are {len(outliers)} outliers')

data_frame.drop(outliers, inplace=True)

In [ ]:
plt.figure(figsize=(15, 7))
plt.title('Distribuzione dei valori di "median_house_value"')

seaborn.distplot(data_frame[column_to_predict])

plt.grid()
plt.show()

###  Missing values
The dataset contains missing values which are handeled by replacing the value with the mean value of the column. This procedure is necessary to avoid errors in the execution.

In [ ]:
for c in data_frame.columns:
    if data_frame[c].hasnans:
        m = data_frame[c].mean()
        data_frame[c].fillna(value=m, inplace=True)

### Categorical feature
Categorical values cannot be treated as such for the statistical analysis, therefore they must be transormed in numbers.

In [ ]:
labelencoder = LabelEncoder()

for c in categories_columns:
    data_frame[c + '_cat'] = labelencoder.fit_transform(data_frame[c])
    
data_frame.drop(columns=categories_columns, inplace=True)

### Correlation of the dataset
Explore the Peasrson's coefficient of correlation and build a simmetric correlation matrix between the features. This procedure is helpeful to eventually reduce the dimension of the dataset. 

Equazioni esempio:

$$ \hat{Y} = \hat{\beta}_{0} + \sum \limits _{j=1} ^{p} X_{j}\hat{\beta}_{j}  $$

In [ ]:
corr = data_frame.corr()

plt.figure(figsize=(15, 15))
plt.title('Correlazione tra le caratteristiche')

seaborn.heatmap(corr, square=True, annot=True)

plt.show()

Pearson's coefficient is determined by :

$$ r_{i,j} = \frac{\sum_{t=1}^m (x_{i,t}-\mu_i)(x_{j,t}-\mu_j)}{\sqrt{\sum_{t=1}^m (x_{i,t}-\mu_i)^2}\sqrt{\sum_{t=1}^m (x_{j,t}-\mu_j)^2}} $$

The coefficient is in between -1 and +1, when it is close to |1| then there is a correlation (positive or negative) otherwise if the coefficient is close to 0 there isn't any correlation. If some features are linearly correlated they are not useful, because we can explain one feature through the correlated one.

In [ ]:
correlated_features = corr[((corr > 0.75) | (corr < -0.75)) & (corr != 1.0)].dropna(axis='index', how='all')
correlated_features

As we can see the pairs of the most correlated features are 
(households - total_bedrooms)
(population - housegìholds)
(total_rooms - total_bedrooms)
(total_bedroom - households)

Delete the correlated features

1. population 
2. total_rooms

In [ ]:
columns_to_remove = ['latitude', 'population', 'total_rooms', column_to_predict]
columns_to_use = list(data_frame.columns)

for u in columns_to_remove:
    columns_to_use.remove(u)
    if numerics_columns.count(u) > 0:
        numerics_columns.remove(u)

y = data_frame[column_to_predict]
X = data_frame.drop(columns=columns_to_remove)

Quindi la nostra attuale X è la seguente:

In [ ]:
X

Quindi la nostra attuale y è la seguente:

In [ ]:
y

----------------
## Scaler
Transform features by scaling each feature to a given range.

This estimator scales and translates each feature individually such that it is in the given range on the training set, e.g. between zero and one.

This transformation is often used as an alternative to zero mean, unit variance scaling.

In [ ]:
column_to_predict_idx = data_frame.columns.get_loc(column_to_predict)
cols = list(range(0, data_frame.shape[1]))
cols.remove(column_to_predict_idx)

scaler = MinMaxScaler()
scaler.fit(data_frame)
data_frame = scaler.transform(data_frame)

X = data_frame[:, cols]
y = data_frame[:, column_to_predict_idx]

----------------
## Learning algorithm
Before starting to learn the algorithm, the dataset must be splitted in subsets in order to preserve some data for the validation of the method.

In [ ]:
cholesky = Cholesky()
svd = SVD()

### Train & Test
Train 60%, Test 40%

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=shuffleDataSet)

### Elaborazione per range di ɑ

In [ ]:
maes_c = []
maes_s = []

r2s_c = []
r2s_s = []

for ɑ in alphas:
    cholesky.alpha = ɑ
    svd.alpha = ɑ

    cholesky.fit(x_train, y_train)
    svd.fit(x_train, y_train)
    
    y_predict_c = cholesky.predict(x_test)
    y_predict_s = svd.predict(x_test)

    # Return the MSE of the prediction.
    mae_c = mean_squared_error(y_predict_c, y_test)
    mae_s = mean_squared_error(y_predict_s, y_test)
    
    maes_c.append(mae_c)
    maes_s.append(mae_s)
    
    r2_c = r2_score(y_predict_c, y_test)
    r2_s = r2_score(y_predict_s, y_test)
    
    r2s_c.append(r2_c)
    r2s_s.append(r2_s)

In [ ]:
plt.figure(figsize=(15, 7))

plt.plot(alphas, maes_c, label='Cholesky')
plt.plot(alphas, maes_s, label='SVD')

plt.xlabel("ɑ")
plt.ylabel("MSE")

plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(15, 7))

plt.plot(alphas, r2s_c, label='Cholesky')
plt.plot(alphas, r2s_s, label='SVD')

plt.xlabel("ɑ")
plt.ylabel("R²")

plt.legend()
plt.grid()
plt.show()

In [ ]:
idx_max_r2_c = numpy.argmax(r2s_c)
idx_max_r2_s = numpy.argmax(r2s_s)

idx_max_mae_c = numpy.argmax(maes_c)
idx_max_mae_s = numpy.argmax(maes_s)

best_alpha_c = alphas[idx_max_r2_c]
best_alpha_s = alphas[idx_max_r2_s]

print(f'Cholesky best ɑ: {best_alpha_c}')
print(f'Cholesky best MAE: {maes_c[idx_max_mae_c]}')
print(f'Cholesky best R²: {maes_c[idx_max_r2_c]}')

print(f'SVD best ɑ: {best_alpha_s}')
print(f'SVD best MAE: {r2s_s[idx_max_r2_s]}')
print(f'SVD best R²: {r2s_c[idx_max_mae_s]}')

In [ ]:
cholesky.alpha = best_alpha_c
svd.alpha = best_alpha_c

cholesky.fit(x_train, y_train)
svd.fit(x_train, y_train)

y_predict_c = cholesky.predict(x_test)
y_predict_s = svd.predict(x_test)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 15))

ax1.set_title('Real vs Predicted - Cholesky')
ax2.set_title('Real vs Predicted - SVD')

seaborn.regplot(y_predict_c, y_test, scatter_kws={'alpha':0.5}, ax=ax1)
seaborn.regplot(y_predict_s, y_test, scatter_kws={'alpha':0.5}, ax=ax2)

ax1.set_xlabel('Predicted')
ax1.set_ylabel('Real')
ax2.set_xlabel('Predicted')
ax2.set_ylabel('Real')

ax1.grid()
ax2.grid()

plt.show()

----------------
## Cross-validation

### Cross-validated risk estimate
Cross Validation is is a method that allows to perform selection and  validation of the learning algorithm. 

Set aside a portion of the data for the test set and perform multiple interations on the training data, splitted in k-folds and validate the hyperparameter to one fold per time . Then compare to the test data.

Method for comparing is the root mean squared error.

Normalize to get values normalized

Use the 5-fold cross validation

#### K-Folds cross-validator
Provides train/test indices to split data in train/test sets.

Split dataset into k consecutive folds (without shuffling by default).

Each fold is then used once as a validation while the k - 1 remaining folds form the training set.

In [ ]:
scores_c = []
scores_s = []

kf = KFold(n_splits=5, shuffle=shuffleDataSet)

for ɑ in alphas:
    k_scores_c = []
    k_scores_s = []
    
    cholesky.alpha = ɑ
    svd.alpha = ɑ
    
    for train_index, test_index in kf.split(X):
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
                
        cholesky.fit(x_train, y_train)
        svd.fit(x_train, y_train)

        y_pred_c = cholesky.predict(x_test)
        y_pred_s = svd.predict(x_test)
        
        # Return the coefficient of determination R² of the prediction.
        score_c = r2_score(y_test, y_pred_s)
        score_s = r2_score(y_test, y_pred_c)

        k_scores_c.append(score_c)
        k_scores_s.append(score_s)
        
    scores_c.append(numpy.mean(k_scores_c))
    scores_s.append(numpy.mean(k_scores_s))

In [ ]:
idx_max_c = numpy.argmax(scores_c)
idx_max_s = numpy.argmax(scores_s)

best_alpha_c = alphas[idx_max_c]
best_alpha_s = alphas[idx_max_s]

print(f'Cholesky best ɑ: {best_alpha_c}')
print(f'Cholesky best score: {scores_c[idx_max_c]}')

print(f'SVD best ɑ: {best_alpha_s}')
print(f'SVD best score: {scores_s[idx_max_s]}')

In [ ]:
plt.figure(figsize=(15, 7))
plt.xlabel('ɑ')
plt.ylabel('R²')

plt.plot(alphas, scores_c, label='Cholesky')
plt.plot(alphas, scores_s, label='SVD')

plt.legend()
plt.grid()
plt.show()

In [ ]:
idx_max_c = numpy.argmax(scores_c)
idx_max_s = numpy.argmax(scores_s)

if best_alpha_c == alphas[idx_max_c]:
    print('Il valore di ɑ per Cholesky è invariato!')
else:    
    best_alpha_c = alphas[idx_max_c]
    print(f'Cholesky best ɑ: {best_alpha_c}')

if best_alpha_s == alphas[idx_max_s]:
    print('Il valore di ɑ per SVD è invariato!')
else:    
    best_alpha_s = alphas[idx_max_s]
    print(f'SVD best ɑ: {best_alpha_s}')

In [ ]:
if best_alpha_c != alphas[idx_max_c]:
    cholesky.set_params(alpha = best_alpha_c)
    cholesky.fit(x_train, y_train, best_alpha_c)
    y_predict_c = cholesky.predict(x_test)
    score_c = mean_squared_error(y_predict_c, y_test)    
    print(f'Cholesky score: {score_c}')

if best_alpha_s != alphas[idx_max_s]:
    svd.set_params(alpha = best_alpha_s)
    svd.fit(x_train, y_train, best_alpha_s)
    y_predict_s = svd.predict(x_test)
    score_s = mean_squared_error(y_predict_s, y_test)
    print(f'SVD score: {score_s}')

In [ ]:
if best_alpha_c != alphas[idx_max_c] or best_alpha_s != alphas[idx_max_s]:
    fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 15))

    ax1.set_title('Real vs Predicted - Cholesky')
    ax2.set_title('Real vs Predicted - SVD')

    seaborn.regplot(y_predict_c, y_test, scatter_kws={'alpha':0.5}, ax=ax1)
    seaborn.regplot(y_predict_s, y_test, scatter_kws={'alpha':0.5}, ax=ax2)

    ax1.set_xlabel('Predicted')
    ax1.set_ylabel('Real')
    ax2.set_xlabel('Predicted')
    ax2.set_ylabel('Real')

    ax1.grid()
    ax2.grid()

    plt.show()

----------------
## Principal component analysis (PCA).

Linear dimensionality reduction using Singular Value Decomposition of the data to project it to a lower dimensional space. The input data is centered but not scaled for each feature before applying the SVD.

In [ ]:
min_ts = int(X.shape[0] * 0.1)
max_ts = int(X.shape[0] * 0.8)
step_ts = int(X.shape[0] * 0.05)
sizes = range(min_ts, max_ts, step_ts)

print(sizes)

In [ ]:
coef_list = []

cholesky.alpha = best_alpha_c

for s in sizes:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=shuffleDataSet)
    cholesky.fit(X_train, y_train)    
    coef_list.append(cholesky.coef_)

coef_matrix = numpy.array(coef_list)

In [ ]:
pca = PCA(n_components=2)
pca.fit(coef_matrix)
coef_pca = pca.transform(coef_matrix)

plt.figure(figsize=(15, 7))
plt.grid()
plt.scatter(coef_pca[:,0], coef_pca[:,1])
plt.show()

In [ ]:
pca = PCA(n_components=X.shape[1])
pca.fit(X)

X_pca = pca.transform(X)

In [ ]:
plt.figure(figsize=(15, 7))
plt.title('PCA')

plt.plot(pca.singular_values_, label='Singular values')

plt.grid()
plt.legend()
plt.show()

We project all the datapoints on the five principal components. Then we check the learning curve again.

In [ ]:
pca = PCA(n_components=5)
pca.fit(X)

X_pca = pca.transform(X)

# Returns:
# -Numbers of training examples that has been used to generate the learning curve. Note that the number of ticks might be less than n_ticks because duplicate entries will be removed;
# -Scores on training sets;
# -Scores on test set.
train_size, train_score, val_score = learning_curve(cholesky, X_pca, y, train_sizes=sizes, cv=5, scoring=scoring, n_jobs=cpu)

train_score_mean = scoring_neg * numpy.mean(train_score, axis=1)
train_score_std = numpy.std(train_score, axis=1)
val_score_mean = scoring_neg * numpy.mean(val_score, axis=1)
val_score_std = numpy.std(val_score, axis=1)

In [ ]:
plt.figure(figsize=(15, 15))
plt.title('PCA Linear Regression')

plt.fill_between(sizes, train_score_mean - train_score_std, train_score_mean + train_score_std, alpha=0.1, color="r")
plt.fill_between(sizes, val_score_mean - val_score_std, val_score_mean + val_score_std, alpha=0.1, color="g")

plt.plot(sizes, train_score_mean, 'o-', color="r", label="Training error")
plt.plot(sizes, val_score_mean, 'o-', color="g", label="CV risk estimate")

plt.xlabel('Training size')
plt.ylabel(scoring_label)

plt.grid()
plt.legend()
plt.show()

In [ ]:
coef_list = []

for s in sizes:
    X_train, X_test, y_train, y_test = train_test_split(X_pca, y, train_size=s, shuffle=shuffleDataSet)
    cholesky.fit(X_train, y_train)
    coef_list.append(cholesky.coef_)
    
coef_matrix = numpy.array(coef_list)

pca = PCA(n_components=2)
pca.fit(coef_matrix)
coef_pca = pca.transform(coef_matrix)

plt.figure(figsize=(15, 7))
plt.grid()
plt.scatter(coef_pca[:,0], coef_pca[:,1])
plt.show()